In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from PIL import Image
# Used to build and deploy machine learning apps
import tensorflow as tf
import keras
# Deep Learning API for creating Neural Networks (Runs on TensorFlow)
from keras import layers
from tensorflow import math
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Activation
from keras.preprocessing.image import ImageDataGenerator

# sklearn : machine learning library
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# Used for accessing files and file names
import pathlib 
import os
import zipfile

In [ ]:
files = '/Users/maniksinghsarmaal/Downloads/s_bin/Sbin_f/dataset'

In [ ]:
def display_random_img(dir,class_name):
    img_dir = dir + '/' + class_name
    files = os.listdir(img_dir)
    
    for i in range(5):
        img_file = random.choice(files)
        img = mpimg.imread(os.path.join(img_dir, img_file))
        plt.imshow(img)
        plt.title(class_name)
        plt.axis("off")
        print(f'Image Shape is: {img.shape}')
        plt.show()

In [ ]:
display_random_img(files+'/train','Organic(1)')

In [ ]:
# Setup train & test directories
train_dir = os.path.join(files,"train")
validation_dir = os.path.join(files,"val")

In [ ]:
# Normalize from 0 to 255 to 0 to 1 
train_IDG = ImageDataGenerator(rescale=1/255)
validation_IDG = ImageDataGenerator(rescale=1/255)

# Load data and turn it into batches
train_data = train_IDG.flow_from_directory(train_dir,
                                                target_size=(256, 256),
                                                batch_size=32,
                                                class_mode="categorical") # Multi-class instead of binary

val_data = validation_IDG.flow_from_directory(validation_dir,
                                                target_size=(256, 256),
                                                batch_size=32,
                                                class_mode="categorical")

In [ ]:
def create_model(model_url,num_classes=3):
    '''
    input:  model url as an input and returns customized model for specified number of classes 
    output: compiled model with Adam optimizer,categorical crossentropy as loss, accuracy as metrics
    '''
    feature_extraction_layer = hub.KerasLayer(model_url,trainable=False,input_shape=(224,224,3))
    
    model=tf.keras.Sequential([
        feature_extraction_layer,
        tf.keras.layers.Dense(num_classes,activation='softmax')
    ])
    return model

In [ ]:
resnet=create_model('https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/classification/versions/1')

In [ ]:
resnet.compile(loss='categorical_crossentropy',
                 optimizer=tf.keras.optimizers.legacy.Adam(),
                 metrics=['accuracy'])

In [ ]:
NUM_EPOCHS = 10

history = resnet.fit(train_data,
                    validation_data=val_data,
                    epochs=NUM_EPOCHS)

In [ ]:
#testing done on test_data
test_dir = os.path.join(files,'test')
test_data = validation_IDG.flow_from_directory(test_dir,
                                                target_size=(256, 256),
                                                batch_size=32,
                                                class_mode="categorical")
resnet.evaluate(test_data)

In [74]:
converter = tf.lite.TFLiteConverter.from_saved_model("resnet_high_smartbin") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

2024-02-19 00:34:23.489356: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-19 00:34:23.489372: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-19 00:34:23.491762: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: resnet_high_smartbin
2024-02-19 00:34:23.506880: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-19 00:34:23.506891: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: resnet_high_smartbin
2024-02-19 00:34:23.562592: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-02-19 00:34:24.179447: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: resnet_high_smartbin
2024-02-19 00:34:24.342207: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. 